In [1]:
# Sequential class
from tensorflow import keras
from keras import layers

In [2]:
# Model creation
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

# Second form
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

In [3]:
try:
    print(model.weights)
except ValueError:
    print("Model has not yet been created i.e. weights have not been created")

Model has not yet been created i.e. weights have not been created


In [4]:
# Calls a model for the first time to build it
model.build(input_shape=(None, 3))
model.weights

[<tf.Variable 'dense_2/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[-0.10926795,  0.27703482, -0.24330038, -0.21311426, -0.08179127,
          0.16265273,  0.2768259 , -0.07245703, -0.29558268, -0.055209  ,
          0.28337598, -0.24476592,  0.27379966,  0.17333537, -0.05596235,
          0.0770095 ,  0.28580475,  0.17886338,  0.25749272,  0.10663608,
          0.17044437, -0.0403457 ,  0.14058518, -0.17720518,  0.26108348,
          0.21224856,  0.2669248 , -0.20871812,  0.09447911, -0.04317781,
         -0.11884663, -0.06586894, -0.24350114,  0.15975624,  0.09519801,
          0.13625938,  0.16209295,  0.04590487, -0.1547206 , -0.1983425 ,
          0.13476986,  0.28039783, -0.01444194, -0.08128706,  0.1855255 ,
         -0.2576043 , -0.28718448,  0.02091315, -0.23625182, -0.10107912,
          0.11269397, -0.03593662,  0.09691501,  0.08848345,  0.11551306,
         -0.173214  ,  0.29470086,  0.2135818 , -0.23313999,  0.0481177 ,
          0.2724951 , -0.01826757,  0.21527

In [5]:
# Now a summary can be written
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                256       
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


In [6]:
#Input shape can be specified in advance for building on the fly
model = keras.Sequential()
model.add(keras.Input(shape=(3,)))
model.add(layers.Dense(64, activation="relu"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                256       
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Two dense layers in the functional API
inputs = keras.Input(shape=(3,), name="my input")
features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)
model = keras.Model(inputs=inputs, outputs=outputs)

In [8]:
# Check shape
print(inputs.shape)
print(features.shape)
print(outputs.shape)

(None, 3)
(None, 64)
(None, 10)


In [9]:
# Multi-input and multi-output model
vocabulary_size = 10000
num_tags = 100
num_departments = 4

# Define model inputs
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

# Define features
features = layers.Concatenate()([title, text_body, tags])
features = layers.Dense(256, activation="relu")(features)
features = layers.Dense(256, activation="relu")(features)
features = layers.Dense(128, activation="relu")(features)
features = layers.Dense(128, activation="relu")(features)
features = layers.Dense(64, activation="relu")(features)

# Outputs
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags],
                    outputs = [priority, department])


In [10]:
# Training model
import numpy as np 

num_samples = 2000

# Dummy data
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

model.compile(optimizer="rmsprop",
            loss=["mean_squared_error", "categorical_crossentropy"],
            metrics=[["mean_absolute_error"], ["accuracy"]])
model.fit([title_data, text_body_data, tags_data],[priority_data, department_data],epochs=50)
model.evaluate([title_data, text_body_data, tags_data], [priority_data, department_data])
priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

Epoch 1/50
63/63 [==============================] - 4s 42ms/step - loss: 93089.5469 - priority_loss: 0.3375 - department_loss: 93089.2031 - priority_mean_absolute_error: 0.5032 - department_accuracy: 0.2675
Epoch 2/50
63/63 [==============================] - 3s 41ms/step - loss: 1752540.0000 - priority_loss: 0.3419 - department_loss: 1752539.7500 - priority_mean_absolute_error: 0.5079 - department_accuracy: 0.2660
Epoch 3/50
63/63 [==============================] - 3s 45ms/step - loss: 10607617.0000 - priority_loss: 0.3419 - department_loss: 10607617.0000 - priority_mean_absolute_error: 0.5079 - department_accuracy: 0.2405
Epoch 4/50
63/63 [==============================] - 3s 41ms/step - loss: 40245764.0000 - priority_loss: 0.3419 - department_loss: 40245764.0000 - priority_mean_absolute_error: 0.5079 - department_accuracy: 0.2420
Epoch 5/50
63/63 [==============================] - 3s 40ms/step - loss: 108187304.0000 - priority_loss: 0.3419 - department_loss: 108187304.0000 - priority

In [11]:
# Multi-input in a subclassed framework
class CustomerTicketModel(keras.Model):

    # Init
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(num_departments, activation="softmax")

    # Forward pass
    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department

# Test
model = CustomerTicketModel(num_departments=4)
priority, department = model({"title": title_data, "text_body": text_body_data, "tags": tags_data})
model.compile(optimizer="rmsprop", loss=["mean_squared_error", "categorical_crossentropy"], 
              metrics=[["mean_absolute_error"], ["accuracy"]])
model.fit({"title": title_data,"text_body": text_body_data,"tags": tags_data},
          [priority_data, department_data], epochs=1)
model.evaluate({"title": title_data,"text_body": text_body_data,"tags": tags_data}, 
               [priority_data, department_data])
priority_preds, department_preds = model.predict({"title": title_data,"text_body": text_body_data,
                                                  "tags": tags_data})


63/63 [==============================] - 0s 3ms/step


In [12]:
# Custom metric
import tensorflow as tf

class customRMS(keras.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(name="total_samples", initializer="zeros", dtype="int32")
    
    def update_state(self, y_true, y_pred, sample_weight=None): 
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)
    
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)


In [13]:
# Toy mnist model for testing
from keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", 
              metrics=["accuracy", customRMS()])
model.fit(train_images, train_labels, epochs=3, validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 6s 4ms/step - loss: 0.2925 - accuracy: 0.9124 - rmse: 7.1842 - val_loss: 0.1499 - val_accuracy: 0.9573 - val_rmse: 7.3630
Epoch 2/3
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1615 - accuracy: 0.9534 - rmse: 7.3494 - val_loss: 0.1159 - val_accuracy: 0.9676 - val_rmse: 7.4016
Epoch 3/3
313/313 [==============================] - 0s 1ms/step - loss: 0.0905 - accuracy: 0.9734 - rmse: 7.4316
